Есть две pandas таблицы, каждая содержит два столбца. В первой это время и сердечный ритм. Во второй — это время и систолическое давление.

Напишите код, который создает третью таблицу, в которой для каждого замера давления в той же строке содержится время и значение ближайшего замера сердечного ритма, если он был сделан обязательно до замера давления и не раньше, чем за 15 минут.

In [1]:
# Произведём импорт необходимых модулей и библиотек

import pandas as pd
from datetime import datetime, timedelta

In [2]:
# Импорт исходных данных

df_hr = pd.DataFrame({'time': [datetime(2022,1,1,7,40), datetime(2022,1,1,9,50), datetime(2022,1,1,10,1)], 'hr': [60, 90,100]})
df_bp = pd.DataFrame({'time': [datetime(2022,1,1,10), datetime(2022,1,1,8)], 'bp': [140, 120]})

In [3]:
# Функция принимает на вход два датафрейма с исходными данными

def welltory_task_1(df_hr, df_bp):
    
    # Произведём слияние df(общий столбец time), отсортируем по убыванию времени, сбросим индексы, чтобы не путаться,
    # заполним пустые значения нулями, вернём тип int64 столбцам hr и bp
    new_df = pd.concat([df_hr, df_bp]) \
               .sort_values('time', ascending = False) \
               .reset_index(drop=True) \
               .fillna(0) \
               .astype({'hr': 'Int64', 'bp': 'Int64'})
    
    # Отберём те значения индексов строк df в список, в которых разница по времени со следующей строкой не превышает 15 минут,
    # имеются показания давления в текущей строке и показания сердечного ритма в следующей
    spisok_bp = [i for i in range(new_df.shape[0] - 1) \
                 if new_df.time[i] - timedelta(minutes = 15) < new_df.time[i + 1] \
                 and new_df.bp[i] > 0 \
                 and new_df.hr[i + 1] > 0]
    
    # Создадим новый df, в котором отберём строки по списку индексов, найденных в предыдущей операции, оставив в нём
    # значения времени и давления, сбросим индекс, переименуем колонку 'time', отразив принадлежность к замеру давления
    bp_df = new_df.iloc[spisok_bp][['time', 'bp']].reset_index(drop=True).rename(columns = {'time': 'time_bp'})
    
    # Информация по сердечному ритму и времени его замера, удовлетворяющая условию задачи,
    # будет содержаться на следующих строках от отобранных нами ранее строк с информацией по давлению
    spisok_hr = [i + 1 for i in spisok_bp]
    
    # Создадим новый df, в котором отберём строки по списку индексов, найденных в предыдущей операции, оставив в нём
    # значения времени и сердечного ритма, сбросим индекс, переименуем колонку 'time', 
    # отразив принадлежность к замеру сердечного ритма
    hr_df = new_df.iloc[spisok_hr][['time', 'hr']].reset_index(drop=True).rename(columns = {'time': 'time_hr'})
    
    # Произведём слияние, полученных df по индексам
    final_df = hr_df.merge(bp_df, left_index = True, right_index = True)
    
    return final_df

In [16]:
welltory_task_1(df_hr, df_bp)

,time_hr,hr,time_bp,bp
0,2022-01-01 09:50:00,90,2022-01-01 10:00:00,140


---

In [4]:
# Импорт исходных данных

df_hr = pd.DataFrame({'time': [datetime(2022,1,1,7,40), datetime(2022,1,1,9,50), datetime(2022,1,1,10,1)], 'hr': [60, 90,100]})
df_bp = pd.DataFrame({'time': [datetime(2022,1,1,10), datetime(2022,1,1,8)], 'bp': [140, 120]})

In [5]:
# df со временем и сердечным ритмом

df_hr

,time,hr
0,2022-01-01 07:40:00,60
1,2022-01-01 09:50:00,90
2,2022-01-01 10:01:00,100


In [6]:
# df со временем и систолическим давлением

df_bp

,time,bp
0,2022-01-01 10:00:00,140
1,2022-01-01 08:00:00,120


In [7]:
# Произведём слияние df(общий столбец time), отсортируем по убыванию времени, сбросим индексы, чтобы не путаться,
# заполним пустые значения нулями, вернём тип int64 столбцам hr и bp

new_df = pd.concat([df_hr, df_bp])
new_df = new_df.sort_values('time', ascending = False).reset_index(drop=True).fillna(0).astype({'hr': 'Int64', 'bp': 'Int64'})
new_df

,time,hr,bp
0,2022-01-01 10:01:00,100,0
1,2022-01-01 10:00:00,0,140
2,2022-01-01 09:50:00,90,0
3,2022-01-01 08:00:00,0,120
4,2022-01-01 07:40:00,60,0


In [8]:
# Отберём те значения индексов строк df в список, в которых разница по времени со следующей строкой не превышает 15 минут,
# имеются показания давления в текущей строке и показания сердечного ритма в следующей

spisok_bp = [i for i in range(new_df.shape[0] - 1) \
             if new_df.time[i] - timedelta(minutes = 15) < new_df.time[i + 1] and new_df.bp[i] > 0 and new_df.hr[i + 1] > 0]

In [9]:
# Создадим новый df, в котором отберём строки по списку индексов, найденных в предыдущей операции, оставив в нём
# значения времени и давления, сбросим индекс, переименуем колонку 'time', отразив принадлежность к замеру давления

bp_df = new_df.iloc[spisok_bp][['time', 'bp']].reset_index(drop=True).rename(columns = {'time': 'time_bp'})
bp_df

,time_bp,bp
0,2022-01-01 10:00:00,140


In [10]:
# Информация по сердечному ритму и времени его замера, удовлетворяющая условию задачи,
# будет содержаться на следующих строках от отобранных нами ранее строк с информацией по давлению

spisok_hr = [i + 1 for i in spisok_bp]

In [11]:
# Создадим новый df, в котором отберём строки по списку индексов, найденных в предыдущей операции, оставив в нём
# значения времени и сердечного ритма, сбросим индекс, переименуем колонку 'time', 
# отразив принадлежность к замеру сердечного ритма

hr_df = new_df.iloc[spisok_hr][['time', 'hr']].reset_index(drop=True).rename(columns = {'time': 'time_hr'})
hr_df

,time_hr,hr
0,2022-01-01 09:50:00,90


In [12]:
# Произведём слияние, полученных df по индексам

final_df = hr_df.merge(bp_df, left_index = True, right_index = True)

In [13]:
# Итог

final_df

,time_hr,hr,time_bp,bp
0,2022-01-01 09:50:00,90,2022-01-01 10:00:00,140


In [14]:
comparison = pd.DataFrame({
'time_hr': [datetime(2022,1,1,9,50)],
'hr': [90],
'time_bp': [datetime(2022,1,1,10)],
'bp': [140]
})

In [15]:
final_df == comparison

,time_hr,hr,time_bp,bp
0,True,True,True,True
